In [33]:
from more_itertools import chunked

In [59]:
import glob

exclude = ['./CLAUDE.md','./progress-tracker.md','./project-description.md']
list_of_files = glob.glob('meridian-islands/**/*.md') 
filtered_list = []
for a in list_of_files:
    if a not in exclude:
        filtered_list.append(a)

filtered_list = sorted(filtered_list, key=lambda p: p.lower())
for a in filtered_list:
  print(a)

markdown_string = ""
for file_name in filtered_list:
    with open(file_name, "r", encoding="utf-8") as f:
        markdown_string += f.read() 

with open("all_info.txt", "w", encoding="utf-8") as text_file:
    text_file.write(markdown_string)

meridian-islands/01-foundation/world-bible.md
meridian-islands/02-geography/islands-geography.md
meridian-islands/03-culture/culture-guide.md
meridian-islands/04-transportation/getting-around.md
meridian-islands/05-accommodations/accommodation-guide.md
meridian-islands/05-accommodations/azure-peak-hotels.md
meridian-islands/05-accommodations/budget-hostels.md
meridian-islands/05-accommodations/coral-bay-hotels.md
meridian-islands/05-accommodations/luxury-hotels.md
meridian-islands/05-accommodations/mid-range-hotels.md
meridian-islands/06-dining/coral-bay-luxury-restaurants.md
meridian-islands/06-dining/coral-bay-restaurants.md
meridian-islands/06-dining/coral-bay-street-food.md
meridian-islands/06-dining/other-islands-dining.md
meridian-islands/07-attractions/coral-bay-attractions.md


In [58]:
splitted_string = markdown_string.split(" ")
chunk = []
for a in range(200):
    chunk.append(splitted_string[a])
print(chunk)

['#', 'Meridian', 'Islands', 'World', 'Bible\n*Master', 'Reference', 'Document', '-', 'All', 'content', 'must', 'align', 'with', 'these', 'established', 'facts*\n\n##', 'Archipelago', 'Overview\n\nThe', 'Meridian', 'Islands', 'are', 'a', 'sovereign', 'island', 'nation', 'located', 'in', 'the', 'South', 'Pacific', 'Ocean,', 'approximately', '2,100', 'kilometers', 'east', 'of', 'Australia', 'and', '1,800', 'kilometers', 'north', 'of', 'New', 'Zealand.', 'The', 'archipelago', 'consists', 'of', 'six', 'islands:', 'four', 'main', 'inhabited', 'islands', 'and', 'two', 'smaller', 'islets.\n\n###', 'The', 'Six', 'Islands\n\n1.', '**Coral', 'Bay', 'Island**', '(385', 'km²)', '-', 'The', 'capital', 'island', 'and', 'main', 'tourist', 'hub\n2.', '**Azure', 'Peak', 'Island**', '(298', 'km²)', '-', 'Adventure', 'and', 'nature', 'tourism', 'center\n3.', '**Amber', 'Cove', 'Island**', '(156', 'km²)', '-', 'Cultural', 'heartland', 'and', 'traditional', 'crafts\n4.', '**Pearl', 'Shoals', 'Island**', '(